**Names** Safyre Anderson, Howard Wen , Vamsi Sakhamuri

**Emails** safyre@berkeley.edu, howard.wen1@gmail.com, vamsi@ischool.berkeley.edu 

**Time of Initial Submission:** March 10th, 2016 8am PST

**Section** W261-3, Spring 2016  

**Week** 7 Homework

==========================================================
## ===HW 7.0: Shortest path graph distances (toy networks)===

In this part of your assignment you will develop the base of your code for the week.

Write MRJob classes to find shortest path graph distances, 
as described in the lectures. In addition to finding the distances, 
your code should also output a distance-minimizing path between the source and target.
Work locally for this part of the assignment, and use 
both of the undirected and directed toy networks.

To proof you code's function, run the following jobs

- shortest path in the undirected network from node 1 to node 4
Solution: 1,5,4 

- shortest path in the directed network from node 1 to node 5
Solution: 1,2,4,5

and report your output---make sure it is correct!

=================================

In [191]:
%%writefile shortest_path.py
from mrjob.job import MRJob
import sys
import ast

# This MrJob computes the shortest paths from a single source
#     Mapper: calculate partial gradient for each example  
#     
class shortest_path(MRJob):
    # run before the mapper processes any input. 
    #Adds state and distance information to all the nodes and creates a new file
                
    # Calculate partial gradient for each example 
    def mapper(self, _,line):
        neighbors = []
        edge_info = []
        node = line.split('\t')
        edges = ast.literal_eval(node[1])
        if(node[4]=='Q'):   ##This node is in the frontier, will be expanded
            edge_info = node[1:-1]
            edge_info.append("V")
            yield node[0],edge_info
            for k in edges.keys():
                if(k!='NULL'):
                    neighbors.append([k,edges[k]])
                
            for n in neighbors:
                yield n[0],[{'NULL':'NULL'},float(node[2])+1,(node[0]+","+node[3]),'Q']
        else:
            yield node[0],node[1:]
                
    def merge(self,n0,n1):
        node_0 = n0
        node_1 = n1
        merged_node = ["OOPS"]
        if(node_0[3]=="Q" and node_1[3]=="U"):
            merged_node = [node_1[0],str(node_0[1]),node_0[2],node_0[3]]
        return merged_node
  
    def reducer_init(self):
        filename = self.options.pathName+"directed_toy_with_state.txt"
        open(filename,"w").close()
        filenamed = self.options.pathName+"debug.txt"
        open(filenamed,"w").close()
    
    def reducer(self,key,line):
        filename = self.options.pathName+"directed_toy_with_state.txt"
        count = 0
        for l in line:
            if(count==0):  
                n0 = l
                count = count+1
                filenamed = self.options.pathName+"debug.txt"
                with open(filenamed,'a+') as fd:
                    fd.write(str(str(key)+str(n0))+"\n")   
            else:
                n1 = l
                count = count+1
                filenamed = self.options.pathName+"debug.txt"
                with open(filenamed,'a+') as fd:
                    fd.write(str(str(key)+str(n1))+"\n")
        if(count==2): #Merging Required
            new = []
            count = 0
            if(n0[3]==u'Q' and n1[3]!=u'V'):
                merged = self.merge(n0,n1)
                with open(filename,'a+') as fw:
                    fw.writelines(key + "\t" + "\t".join(merged) + "\n")
                yield key,merged
            elif(n1[3]==u'Q' and n0[3]!=u'V'):
                merged = self.merge(n1,n0)
                with open(filename,'a+') as fw:
                    fw.writelines(key + "\t" + "\t".join(merged) + "\n")
                yield key,merged
            elif(n0[3]==u'V'):
                new.append(str(key))
                for n in n0:
                    new.append(str(n))
                with open(filename,'a+') as fw:
                    fw.writelines("\t".join(new) + "\n")
                yield key,n0
            elif(n1[3]==u'V'):
                new.append(str(key))
                for n in n1:
                    new.append(str(n))
                with open(filename,'a+') as fw:
                    fw.writelines("\t".join(new) + "\n")
                yield key,n1                
        else:  #No merging required
            count = 0
            new = []
            new.append(str(key))
            for n in n0:
                new.append(str(n))
            with open(filename,'a+') as fw:
                fw.writelines("\t".join(new)+"\n")
            yield key,n0
 
    def configure_options(self):
        super(shortest_path, self).configure_options()
        self.add_passthrough_option(
            '--pathName', dest='pathName', default="", type='str',
            help='pathName: pathname where intermedateResults.txt is stored')
    
    def steps(self):
        return [self.mr(mapper=self.mapper,reducer_init=self.reducer_init,reducer=self.reducer)] 
    
if __name__ == '__main__':
    shortest_path.run()

Overwriting shortest_path.py


In [192]:
%reload_ext autoreload
%autoreload 2

from numpy import random,array
from shortest_path import shortest_path
import sys

first = True

#Transforming the directed_toy.txt to included state,distance,path information
with open('directed_toy.txt', 'r') as fr:
    for f in fr:
        adj_list = []
        adjl = f.strip().split('\t')
        if(first==True):
            adj_list.append(adjl[0]) #Node
            adj_list.append(adjl[1]) #Edges and Weights
            adj_list.append(str(0))  #Shortest Distance
            adj_list.append('NA')    #Shortest Path          
            adj_list.append('Q')     #state
            first=False
        else:
            adj_list.append(adjl[0])         #Node
            adj_list.append(adjl[1])         #Edges and Weights
            adj_list.append(str(sys.maxint)) #Shortest Distance
            adj_list.append('NA')            #Shortest Path
            adj_list.append('U')             #state
        
        with open('directed_toy_with_state.txt','a+') as fw:
            fw.writelines('\t'.join(adj_list)+'\n')

mr_job = shortest_path(args=['directed_toy_with_state.txt','--pathName','/Users/Vamsi/Documents/W261/hw7/']) 

i = 0 
while(1):
    print "iteration ",i
    unvisited_nodes = 0
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key,value
            if(value[3]!=u'V'):
                unvisited_nodes = unvisited_nodes + 1
            
    if(unvisited_nodes==0):
        break
    i +=1

print "Done- All nodes have been visited"
            

iteration  0
1

 [u"{'2': 1, '6': 1}", u'0', u'NA', u'V']
2 [u"{'1': 1, '3': 1, '4': 1}", u'1.0', u'1,NA', u'Q']
3 [u"{'2': 1, '4': 1}", u'9223372036854775807', u'NA', u'U']
4 [u"{'2': 1, '5': 1}", u'9223372036854775807', u'NA', u'U']
5 [u"{'1': 1, '2': 1, '4': 1}", u'9223372036854775807', u'NA', u'U']
6 [{u'NULL': u'NULL'}, 1.0, u'1,NA', u'Q']
iteration  1
1

 [u"{'2': 1, '6': 1}", u'0', u'NA', u'V']
2 [u"{'1': 1, '3': 1, '4': 1}", u'1.0', u'1,NA', u'V']
3 [u"{'2': 1, '4': 1}", u'2.0', u'2,1,NA', u'Q']
4 [u"{'2': 1, '5': 1}", u'2.0', u'2,1,NA', u'Q']
5 [u"{'1': 1, '2': 1, '4': 1}", u'9223372036854775807', u'NA', u'U']
6 [u"{u'NULL': u'NULL'}", u'1.0', u'1,NA', u'V']
iteration  2
1

 [u"{'2': 1, '6': 1}", u'0', u'NA', u'V']
2 [u"{'1': 1, '3': 1, '4': 1}", u'1.0', u'1,NA', u'V']
3 [u"{'2': 1, '4': 1}", u'2.0', u'2,1,NA', u'V']
4 [u"{'2': 1, '5': 1}", u'2.0', u'2,1,NA', u'V']
5 [u"{'1': 1, '2': 1, '4': 1}", u'3.0', u'4,2,1,NA', u'Q']
6 [u"{u'NULL': u'NULL'}", u'1.0', u'1,NA', u'V']
iteration  3
1 [u"{'2': 1, '6': 1}", u'0', u'NA', u'V']
2 [u"{'1': 1, '3': 1, '4': 1}", u'1.0', u'1,NA', u'V']
3 [u"{'2': 1, '4': 1}", u'2.0', u'2,1,NA', u'V']
4 [u"{'2': 1, '5': 1}", u'2.0', u'2,1,NA', u'V']
5 [u"{'1': 1, '2': 1, '4': 1}", u'3.0', u'4,2,1,NA', u'V']
6 [u"{u'NULL': u'NULL'}", u'1.0', u'1,NA', u'V']
Done- All nodes have been visited
